In [ ]:
import os
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from typing import Optional



def start_browser(webdriver_path: Optional[str] = None) -> webdriver.Chrome:
    """
    Selenium WebDriver(Chrome) 객체를 생성하여 브라우저를 시작합니다.
    시스템 기본 path에 webdriver가 있을 경우 webdriver_path를 None으로 두면 됩니다.
    options 에서 필요에 따라 option을 추가하세요.
    
    Args:
        webdriver_path (Optional[str]): 크롬드라이버 경로. 기본값은 None이며,
                                        시스템 환경 변수 설정을 사용할 경우 None으로 두면 됩니다.
    
    Returns:
        webdriver.Chrome: 생성된 Chrome WebDriver 객체.
    """
    # Chrome WebDriver 옵션 설정
    options = Options()


    # Headless 모드: 브라우저 창을 띄우지 않고 백그라운드에서 실행합니다.
    # options.add_argument("--headless")

    # --no-sandbox: 샌드박스 모드를 비활성화합니다.
    # 일부 리눅스 환경이나 컨테이너 환경에서 권한 문제를 피하기 위해 사용합니다.
    options.add_argument("--no-sandbox")

    # --disable-dev-shm-usage: /dev/shm(공유 메모리) 사용을 비활성화합니다.
    # Docker와 같이 /dev/shm 용량이 제한된 환경에서 메모리 부족 문제를 방지하기 위해 사용됩니다.
    options.add_argument("--disable-dev-shm-usage")

    # --disable-blink-features=AutomationControlled: 자동화 제어 플래그를 비활성화하여
    # 웹사이트에서 Selenium을 통한 자동화 제어 여부를 감지하지 못하도록 합니다.
    options.add_argument("--disable-blink-features=AutomationControlled")

    # user-agent 설정: 서버에 전달되는 브라우저 식별 문자열(User Agent)을 변경합니다.
    # 실제 사용자 브라우저처럼 보이게 하여, 자동화 스크립트임을 감추기 위해 사용합니다.
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )

    # --lang=ko_KR: 브라우저의 기본 언어를 한국어로 설정합니다.
    # 웹사이트가 언어 설정에 따라 다르게 동작할 경우, 한국어 환경으로 테스트할 수 있습니다.
    options.add_argument("--lang=ko_KR")

    # --charset=utf-8: 문자 인코딩을 UTF-8로 설정하려는 시도입니다.
    # (참고: 이 옵션은 크롬에서 공식적으로 지원하지 않을 수 있으므로, 실제 효과는 미미할 수 있습니다.)
    options.add_argument("--charset=utf-8")
    
    # WebDriver 객체 생성
    if webdriver_path:
        service = Service(webdriver_path)
    else:
        service = Service()  # 시스템 환경 변수에 설정된 경로를 사용
    
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def append_to_json(json_file, new_data):
    """
    json_file에 기존 데이터가 있다면 불러와 new_data를 업데이트한 후 저장합니다.
    """
    if os.path.exists(json_file):
        with open(json_file, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = {}
    else:
        data = {}
    
    data.update(new_data)
    
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def scrap_list(driver, json_file):
    """
    base_url를 기준으로 page=1부터 page=100까지 반복하며 각 페이지에서 최대 100개의 책 링크와 제목을 수집합니다.
    수집된 데이터는 {index: [책 제목, 링크]} 형태의 딕셔너리로 JSON 파일에 이어붙입니다.
    
    Args:
        driver: Selenium WebDriver 객체.
        base_url: 시작 URL (예: "https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1&ref=ls_pl_car_0")
        json_file: 데이터를 저장할 JSON 파일 경로.
    """
    global_index = 1  # 전체 URL에 대한 인덱스
    
    for page in range(1, 101):
        # base_url의 '?' 이전 부분을 취한 후, page 파라미터와 ref를 새로 추가하여 URL 구성
        
        page_url = f"https://www.goodreads.com/list/show/1.Best_Books_Ever?page={page}&ref=ls_pl_car_0"

        print(f"{page}현재 페이지 처리 중")
        driver.get(page_url)
        time.sleep(3)
        
        page_data = {}
        # 각 페이지에서 최대 100개의 책 링크와 제목 수집
        for i in range(1, 101):
            xpath_link = f"//*[@id='all_votes']/table/tbody/tr[{i}]/td[3]/a"
            xpath_book = f"//*[@id='all_votes']/table/tbody/tr[{i}]/td[3]/a/span"
            try:
                elem = driver.find_element(By.XPATH, xpath_link)
                link = elem.get_attribute("href")
                
                # 책 제목을 가져옵니다.
                book_elem = driver.find_element(By.XPATH, xpath_book)
                book_title = book_elem.text
                
                # global_index를 키로 하여 [책 제목, 링크] 형태로 저장
                page_data[global_index] = [book_title, link]
                global_index += 1
            except Exception as e:
                print(f"행 {i}의 책 링크 추출 실패: {e}")
                continue
        
        # 현재 페이지에서 수집한 데이터를 JSON 파일에 이어붙임
        append_to_json(json_file, page_data)

if __name__ == "__main__":

    # 저장할 JSON 파일 경로
    json_file_path = "database/book_links.json"
    
    driver = start_browser()  # webdriver_path를 전달하지 않으면 시스템 환경 변수 사용
    try:
        scrap_list(driver, json_file_path)
    except Exception as e:
        print("크롤링 중 오류 발생:", e)
    finally:
        driver.quit()

In [5]:
import subprocess

webdriver_path = r"C:\Users\user\.cache\selenium\chromedriver\win64\131.0.6778.264\chromedriver.exe"
version_info = subprocess.check_output([webdriver_path, '--version'])
print(version_info.decode('utf-8').strip())


ChromeDriver 131.0.6778.264 (2d05e31515360f4da764174f7c448b33e36da871-refs/branch-heads/6778@{#4323})


In [2]:
import sys
print(sys.path)


['c:\\Users\\user\\anaconda3\\envs\\singip\\python312.zip', 'c:\\Users\\user\\anaconda3\\envs\\singip\\DLLs', 'c:\\Users\\user\\anaconda3\\envs\\singip\\Lib', 'c:\\Users\\user\\anaconda3\\envs\\singip', '', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\Users\\user\\anaconda3\\envs\\singip\\Lib\\site-packages']
